In [1]:
import pandas as pd
import numpy as np
df = pd.read_csv('merged_data.csv')
df = pd.get_dummies(df, columns = ['HourlySkyConditions'])
df.head()

,DATE,Total Power,HourlyDewPointTemperature,HourlyDryBulbTemperature,HourlyPrecipitation,HourlyRelativeHumidity,HourlyStationPressure,HourlyVisibility,HourlyWetBulbTemperature,HourlyWindDirection,...,RA,SN,TS,VCTS,HourlySkyConditions_BKN,HourlySkyConditions_CLR,HourlySkyConditions_FEW,HourlySkyConditions_OVC,HourlySkyConditions_SCT,HourlySkyConditions_VV
0,3/1/19 0:00,0.0,9,17,0,70,29.64,10,15,130,...,0,0,0,0,0,0,0,1,0,0
1,3/1/19 1:00,0.0,9,17,0,70,29.63,10,15,120,...,0,0,0,0,0,0,0,1,0,0
2,3/1/19 2:00,0.0,9,17,0,70,29.62,10,15,120,...,0,0,0,0,0,0,0,1,0,0
3,3/1/19 3:00,0.0,10,19,0,68,29.63,10,17,90,...,0,0,0,0,0,0,0,1,0,0
4,3/1/19 4:00,0.0,8,16,0,71,29.63,10,14,120,...,0,0,0,0,0,0,0,0,1,0


In [2]:
for i, row in df.iterrows():
    for column in df.columns[1:]:
        try:
            df.at[i, column] = df.at[i, column].astype(float)
        except:
            df.at[i, column] = 0

In [3]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, precision_score, recall_score, roc_auc_score
from sklearn import metrics
from sklearn.model_selection import train_test_split

In [4]:
X = df.drop(columns = ['Total Power', 'DATE'])
Y = df['Total Power']

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state=12345)

print(X_train.shape, X_test.shape, Y_train.shape, Y_test.shape)

(7027, 29) (1757, 29) (7027,) (1757,)


In [5]:
rf = RandomForestRegressor()
rf.fit(X_train, Y_train)

print(100* rf.score(X_test, Y_test))
print(metrics.mean_absolute_error(Y_test, rf.predict(X_test)))

92.05004716137833
984.0524027230924


In [6]:
pd.options.display.float_format = '{:.4f}'.format
importance = pd.DataFrame({'col_name': rf.feature_importances_}, index=df.drop(columns = ['Total Power', 'DATE']).columns).sort_values(by='col_name', ascending=False)
importance['col_name'] = 100*importance['col_name']
importance

,col_name
HourlyWetBulbTemperature,91.6407
HourlyDryBulbTemperature,2.0135
HourlyWindSpeed,2.0091
HourlyRelativeHumidity,1.5475
HourlyDewPointTemperature,1.2159
HourlySkyConditions_OVC,0.4461
HourlySkyConditions_BKN,0.2594
HourlySkyConditions_CLR,0.2124
HourlySkyConditions_SCT,0.1970
HourlySkyConditions_FEW,0.1834


In [7]:
from datetime import datetime
datetime.strptime(df['DATE'][0], "%m/%j/%y %H:%M")

datetime.datetime(2019, 1, 1, 0, 0)

In [8]:
hours = []
for i, row in df.iterrows():
    hours.append(datetime.strptime(row['DATE'], '%m/%d/%y %H:%M').hour)
    
df['hour'] = hours 

In [9]:
X = df.drop(columns = ['Total Power', 'DATE'])
Y = df['Total Power']

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state=12345)

print(X_train.shape, X_test.shape, Y_train.shape, Y_test.shape)

rf = RandomForestRegressor()
rf.fit(X_train, Y_train)

print(100* rf.score(X_test, Y_test))
print(metrics.mean_absolute_error(Y_test, rf.predict(X_test)))

importance = pd.DataFrame({'col_name': rf.feature_importances_}, index=df.drop(columns = ['Total Power', 'DATE']).columns).sort_values(by='col_name', ascending=False)
importance['col_name'] = 100*importance['col_name']
importance

(7027, 30) (1757, 30) (7027,) (1757,)
93.28370201012387
893.5738171967492


,col_name
HourlyWetBulbTemperature,91.6170
hour,1.7698
HourlyDryBulbTemperature,1.6736
HourlyWindSpeed,1.4594
HourlyRelativeHumidity,1.1903
HourlyDewPointTemperature,0.9806
HourlySkyConditions_OVC,0.4132
HourlySkyConditions_BKN,0.2048
HourlySkyConditions_CLR,0.1722
HourlySkyConditions_FEW,0.1529


In [10]:
df['Total Power'].value_counts()

0.0000       2036
2977.4844     167
2130.4844      48
2582.5938       2
1263.7188       2
             ... 
1295.4767       1
3532.8135       1
9742.4790       1
1963.5781       1
7488.8481       1
Name: Total Power, Length: 6518, dtype: int64

In [11]:
weekdays = []
for i, row in df.iterrows():
    weekdays.append(datetime.strptime(row['DATE'], '%m/%d/%y %H:%M').weekday())
    
df['weekday'] = weekdays

df = pd.get_dummies(df, columns = ['weekday'])

In [12]:
X = df.drop(columns = ['Total Power', 'DATE'])
Y = df['Total Power']

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state=12345)

print(X_train.shape, X_test.shape, Y_train.shape, Y_test.shape)

rf = RandomForestRegressor()
rf.fit(X_train, Y_train)

print(100* rf.score(X_test, Y_test))
print(metrics.mean_absolute_error(Y_test, rf.predict(X_test)))

importance = pd.DataFrame({'col_name': rf.feature_importances_}, index=df.drop(columns = ['Total Power', 'DATE']).columns).sort_values(by='col_name', ascending=False)
importance['col_name'] = 100*importance['col_name']
importance

(7027, 37) (1757, 37) (7027,) (1757,)
95.01991353458142
754.1774912254573


,col_name
HourlyWetBulbTemperature,91.4508
HourlyDryBulbTemperature,1.6160
hour,1.4384
HourlyWindSpeed,1.1513
HourlyRelativeHumidity,0.9156
HourlyDewPointTemperature,0.8295
HourlySkyConditions_OVC,0.3981
weekday_6,0.3158
weekday_2,0.2391
weekday_1,0.2177
